# Data Cleaning in Pandas 1

Data cleaning is one of the fundamental tasks for Data professionals. Here, I have presented with some basic cleaning/wrangling operations. The dataset is sales data and contains information like ID, date of birth (dob), gender, city, last purchase date and income(in $ amount).

In [23]:
# Import necessary libraries
import pandas as pd

In [24]:
# check pandas version
print(pd.__version__)

2.0.3


### Task:
1. Load the raw data file into a pandas DataFrame. 
2. Check the data for missing values and duplicated records. 
3. Remove any duplicate records from the data. 
4. Fill in any missing values in the gender, marital_status, and city columns with the mode of the respective columns. 
5. Create a new column named "age" that contains the age of each customer based on their date of birth. 
6. Create a new column named "income_group" that categorizes customers into three groups based on their income amount- "Low",      "Medium", "High" based on each 33% percentile.
7. Create another column "score_group" that categorizes customers into three groups based on their score -"Poor", "Fair", and      "Good" based on each 33% percentile. 
8. Keep only the records of 2018 and beyonds. 
9. The ID column here is rather long.Shorten it, keep only the last ten characters.
10. Save the cleaned data to a new CSV file named sales_demo_final_data.csv. 


### My Approach:

To prepare the data based on the above instructions, I followed the below process. First, I created a function "dataprep", and inside this function, I make all the operations. So, the functio returns a cleaned dataset.

In [25]:
def dataprep(filename):
    
    # Read the data in a .csv file
    df = pd.read_csv(filename)
    
    # Remove duplicate rows
    df.drop_duplicates(inplace=True)
    
    # Keeping only the last ten characters of the ID 
    df['id'] = df['id'].str[:10]
    
    # converting dob to datetime
    df['dob'] = pd.to_datetime(df['dob'])
    
    
    
    # Creating age column
    df['age'] = ((pd.to_datetime('today') - df['dob']).dt.days/365.2425).floordiv(1)
    
   
    
    # Find out the modes for gender, marital_status and city. Taking the first mode in case of multiple modes
    mode_city = df['city'].mode().values[0]
    mode_gender = df['gender'].mode().values[0]
    mode_marital_status = df['marital_status'].mode().values[0]
    
    #filling the missing values of the above columns by their modes
    
    df['city'].fillna(mode_city, inplace=True)
    df['gender'].fillna(mode_gender, inplace=True)
    df['marital_status'].fillna(mode_marital_status, inplace=True)
    
    # Creating income groups using qcut to divide the column into three groups
    df['income_group'] = pd.qcut(df['income'], q=[0, 0.33, 0.67, 1], labels=['Low', 'Medium', 'High'])
    
    # Creating score groups
    df['score_group'] = pd.qcut(df['score'], q = [0, 0.33, 0.67, 1], labels=['Poor', 'Fair', 'Good'])
    
    # converting the last_purchase_date to a date column
    df['last_purchase_date'] = pd.to_datetime(df['last_purchase_date'], format='mixed')
    
    # Masking for the purchase date 2018 and beyond
    mask_purchase_date = df['last_purchase_date'].dt.year > 2017
    
    # filtering the data according to the above mask
    df = df[mask_purchase_date]
    
   
    return df

In [26]:
df = dataprep('raw_data.csv')
df.head()

,id,gender,dob,income,marital_status,city,last_purchase_date,score,age,income_group,score_group
0,660ba2ad-e,Male,1977-05-19,46532,Single,Oklahoma City,2020-06-22,63,46.0,Medium,Poor
1,6da25e92-a,Female,2000-11-28,13734,Single,Columbus,2020-08-04,43,22.0,Low,Poor
2,11261389-d,Female,1975-06-05,36282,Single,Miami,2022-08-21,882,48.0,Medium,Good
3,c9f67b5a-f,Male,1992-05-27,83451,Divorced,Colorado Springs,2020-12-18,653,31.0,High,Fair
4,d3024d83-c,Female,1987-09-13,58351,Divorced,New Orleans,2021-05-25,535,36.0,Medium,Fair


In [27]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 38573 entries, 0 to 984525
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  38573 non-null  object        
 1   gender              38573 non-null  object        
 2   dob                 38573 non-null  datetime64[ns]
 3   income              38573 non-null  int64         
 4   marital_status      38573 non-null  object        
 5   city                38573 non-null  object        
 6   last_purchase_date  38573 non-null  datetime64[ns]
 7   score               38573 non-null  int64         
 8   age                 38573 non-null  float64       
 9   income_group        38573 non-null  category      
 10  score_group         38573 non-null  category      
dtypes: category(2), datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 3.0+ MB
None


In [17]:
# Saving the cleaned data as a .csv file 
df.to_csv('sales_demo_final_data.csv', index=False)